In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf

import cv2
import os
import PIL

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential 

In [ ]:
data_dir = '/content/drive/MyDrive/dataset1'

In [ ]:
import pathlib 
data_dir = pathlib.Path(data_dir)
data_dir

PosixPath('/content/drive/MyDrive/dataset1')

In [ ]:
xray_dict = {
    'COVID': list(data_dir.glob('COVID/*.png')),
    'Normal': list(data_dir.glob('Normal/*.png')),
    'Viral Pneumonia': list(data_dir.glob('Viral Pneumonia/*.png')),
}

In [ ]:
xray_label_dict = {
    'COVID': 0,
    'Normal': 1,
    'Viral Pneumonia': 2
}

In [ ]:
import cv2 as cv
import numpy as np

In [ ]:
X,Y = [],[]
#resizing the images
i = 0
for disease_name,images in xray_dict.items():
    for image in images:
        img = cv2.imread(str(image))
        i +=1
        print(disease_name+str(i))
        resized_img = cv2.resize(img,(224,224))
       # global standardization of pixels
        X.append(resized_img)
        Y.append(xray_label_dict[disease_name])

Streaming output truncated to the last 5000 lines.
COVID1001
COVID1002
COVID1003
COVID1004
COVID1005
COVID1006
COVID1007
COVID1008
COVID1009
COVID1010
COVID1011
COVID1012
COVID1013
COVID1014
COVID1015
COVID1016
COVID1017
COVID1018
COVID1019
COVID1020
COVID1021
COVID1022
COVID1023
COVID1024
COVID1025
COVID1026
COVID1027
COVID1028
COVID1029
COVID1030
COVID1031
COVID1032
COVID1033
COVID1034
COVID1035
COVID1036
COVID1037
COVID1038
COVID1039
COVID1040
COVID1041
COVID1042
COVID1043
COVID1044
COVID1045
COVID1046
COVID1047
COVID1048
COVID1049
COVID1050
COVID1051
COVID1052
COVID1053
COVID1054
COVID1055
COVID1056
COVID1057
COVID1058
COVID1059
COVID1060
COVID1061
COVID1062
COVID1063
COVID1064
COVID1065
COVID1066
COVID1067
COVID1068
COVID1069
COVID1070
COVID1071
COVID1072
COVID1073
COVID1074
COVID1075
COVID1076
COVID1077
COVID1078
COVID1079
COVID1080
COVID1081
COVID1082
COVID1083
COVID1084
COVID1085
COVID1086
COVID1087
COVID1088
COVID1089
COVID1090
COVID1091
COVID1092
COVID1093
COVID1094
COVID1095

In [ ]:
len(Y)

6000

In [ ]:
X = np.array(X)
Y = np.array(Y)

In [ ]:
X[0]

array([[[-2.03387591, -2.03387591, -2.03387591],
        [-2.31198692, -2.31198692, -2.31198692],
        [-2.49739426, -2.49739426, -2.49739426],
        ...,
        [-2.57155719, -2.57155719, -2.57155719],
        [-2.47885352, -2.47885352, -2.47885352],
        [-2.33052765, -2.33052765, -2.33052765]],

       [[-2.03387591, -2.03387591, -2.03387591],
        [-2.29344619, -2.29344619, -2.29344619],
        [-2.47885352, -2.47885352, -2.47885352],
        ...,
        [-2.55301646, -2.55301646, -2.55301646],
        [-2.46031279, -2.46031279, -2.46031279],
        [-2.29344619, -2.29344619, -2.29344619]],

       [[-2.01533518, -2.01533518, -2.01533518],
        [-2.27490545, -2.27490545, -2.27490545],
        [-2.44177206, -2.44177206, -2.44177206],
        ...,
        [-2.53447572, -2.53447572, -2.53447572],
        [-2.42323132, -2.42323132, -2.42323132],
        [-2.27490545, -2.27490545, -2.27490545]],

       ...,

       [[ 0.20955286,  0.20955286,  0.20955286],
        [ 0

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.25,stratify=Y)

In [ ]:
from tensorflow.keras.applications.resnet import ResNet101
from tensorflow.keras.applications.resnet import preprocess_input
def build_model1(hp):
  resnet101 = ResNet101(input_shape=(224,224,3),weights='imagenet',include_top=False)

  for layer in resnet101.layers:
    layer.trainable = False

  for index in range(hp.Int('num_conv2D',1,4)):
    x = tf.keras.layers.Conv2D(filters=64,kernel_size=hp.Choice('conv1_kernel',values=[2,3]),padding='same',activation='relu')(resnet101.output)
    x = tf.keras.layers.Dropout(rate=hp.Choice('dropoutinside1',values=[0.1,0.2,0.3]))(x)
    
  x = tf.keras.layers.MaxPooling2D(pool_size=hp.Choice('pooling1',values=[3]),padding = 'same')(x)
  x = tf.keras.layers.Dropout(rate=hp.Choice('dropoutoutside1',values=[0.1,0.2,0.3]))(x)
  
  for index in range(hp.Int('num_conv2D_1',1,4)):
    x = tf.keras.layers.Conv2D(filters=128,kernel_size=hp.Choice('conv2_kernel',values=[2,3]),padding='same',activation='relu')(resnet101.output)
    x = tf.keras.layers.Dropout(rate=hp.Choice('dropoutinside2',values=[0.1,0.2,0.3]))(x)
   
  x = tf.keras.layers.MaxPooling2D(pool_size=hp.Choice('pooling2',values=[3]),padding = 'same')(x)
  x = tf.keras.layers.Dropout(rate=hp.Choice('dropoutoutside2',values=[0.1,0.2,0.3]))(x)
 
  for index in range(hp.Int('num_conv2D_2',1,4)):
    x = tf.keras.layers.Conv2D(filters=256,kernel_size=hp.Choice('conv3_kernel',values=[2,3]),padding='same',activation='relu')(resnet101.output)
    x = tf.keras.layers.Dropout(rate=hp.Choice('dropoutinside3',values=[0.1,0.2,0.3]))(x)
  
  x = tf.keras.layers.MaxPooling2D(pool_size=hp.Choice('pooling3',values=[3]),padding = 'same')(x)
  x = tf.keras.layers.Dropout(rate=hp.Choice('dropoutoutside3',values=[0.1,0.2,0.3]))(x)

  x = tf.keras.layers.Flatten()(x)
  for i in range(hp.Int('num_layers', 1, 5)):
    x = tf.keras.layers.Dense(units=hp.Int('units' + str(i),min_value=32,max_value=512,step=32),activation='relu')(x)
    x = tf.keras.layers.Dropout(rate=hp.Choice('dropout3',values=[0.1,0.2,0.3]))(x)
    
  x = tf.keras.layers.Dense(3,activation='softmax')(x)
  
  model = Model(inputs=resnet101.input,outputs=x)
  model.compile(optimizer=tf.keras.optimizers.Adam(hp.Choice('learning_rate',values=[1e-4])),loss=tf.keras.losses.SparseCategoricalCrossentropy(),metrics=['accuracy'])

  return model

In [ ]:
pip install -U keras-tuner

     |████████████████████████████████| 71kB 8.0MB/s 
  Created wheel for keras-tuner: filename=keras_tuner-1.0.2-cp37-none-any.whl size=78938 sha256=3759bd0bfdbd080a4bc40a7924d3efa30f8ddeb58c8051e52a6c7efd0d7f655d
  Stored in directory: /root/.cache/pip/wheels/bb/a1/8a/7c3de0efb3707a1701b36ebbfdbc4e67aedf6d4943a1f463d6
  Created wheel for terminaltables: filename=terminaltables-3.1.0-cp37-none-any.whl size=15356 sha256=5c4bf5317aba219be0727abfe3275dd15c69bdc6120265760aa8ed3cde74dacd
  Stored in directory: /root/.cache/pip/wheels/30/6b/50/6c75775b681fb36cdfac7f19799888ef9d8813aff9e379663e
Successfully built keras-tuner terminaltables


In [ ]:
from kerastuner.tuners import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

ModuleNotFoundError: ignored

In [ ]:
from tensorflow.keras.models import Model

In [ ]:
tuner_search4 = RandomSearch(build_model1,objective='val_accuracy',max_trials=5,directory='output5')

In [ ]:
tuner_search4.search(X_train,Y_train,epochs=10,validation_data=(X_test,Y_test))

Trial 5 Complete [00h 11m 17s]
val_accuracy: 0.9673333168029785

Best val_accuracy So Far: 0.9713333249092102
Total elapsed time: 00h 56m 25s
INFO:tensorflow:Oracle triggered exit


In [ ]:
model1 = tuner_search4.get_best_models(1)[0]

In [ ]:
model1.save('best_model.h5')

In [ ]:
model1.evaluate(X_test,Y_test)

47/47 [==============================] - 16s 345ms/step - loss: 0.1016 - accuracy: 0.9713


[0.1015576496720314, 0.9713333249092102]

In [ ]:
print(tuner_search4.get_best_hyperparameters()[0].values)

{'num_conv2D': 2, 'conv1_kernel': 2, 'dropoutinside1': 0.2, 'pooling1': 3, 'dropoutoutside1': 0.3, 'num_conv2D_1': 3, 'conv2_kernel': 2, 'dropoutinside2': 0.3, 'pooling2': 3, 'dropoutoutside2': 0.3, 'num_conv2D_2': 3, 'conv3_kernel': 2, 'dropoutinside3': 0.2, 'pooling3': 3, 'dropoutoutside3': 0.1, 'num_layers': 2, 'units0': 512, 'dropout3': 0.2, 'learning_rate': 0.0001, 'units1': 512, 'units2': 448, 'units3': 192}


Resnet101


In [ ]:
from tensorflow.keras.applications.resnet import ResNet101
from tensorflow.keras.applications.resnet import preprocess_input
resnet101 = ResNet101(input_shape=(224,224,3),weights='imagenet',include_top=False)

for layer in resnet101.layers:
    layer.trainable = False

x = tf.keras.layers.Flatten()(resnet101.output)
pred1 = tf.keras.layers.Dense(3,activation='softmax')(x)

from tensorflow.keras.models import Model
model = Model(inputs=resnet101.input,outputs=pred1)
model.compile(optimizer='adam',loss=tf.keras.losses.SparseCategoricalCrossentropy(),metrics=['accuracy'])
model.fit(X_train,Y_train,batch_size=32,epochs=10)
model.evaluate(X_test,Y_test)

Epoch 1/10
141/141 [==============================] - 55s 340ms/step - loss: 4.3552 - accuracy: 0.8270
Epoch 2/10
141/141 [==============================] - 48s 342ms/step - loss: 0.4184 - accuracy: 0.9713
Epoch 3/10
141/141 [==============================] - 48s 342ms/step - loss: 0.2271 - accuracy: 0.9801
Epoch 4/10
141/141 [==============================] - 48s 342ms/step - loss: 0.1430 - accuracy: 0.9910
Epoch 5/10
141/141 [==============================] - 48s 343ms/step - loss: 0.1124 - accuracy: 0.9919
Epoch 6/10
141/141 [==============================] - 48s 342ms/step - loss: 0.0805 - accuracy: 0.9937
Epoch 7/10
141/141 [==============================] - 48s 342ms/step - loss: 0.0462 - accuracy: 0.9962
Epoch 8/10
141/141 [==============================] - 48s 342ms/step - loss: 0.2948 - accuracy: 0.9868
Epoch 9/10
141/141 [==============================] - 48s 342ms/step - loss: 0.0450 - accuracy: 0.9963
Epoch 10/10
47/47 [==============================] - 18s 342ms/step - los

[2.226936101913452, 0.9506666660308838]

VGG16


In [ ]:
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input
vgg16 = VGG16(input_shape=(224,224,3),weights='imagenet',include_top=False)

for layer in vgg16.layers:
    layer.trainable = False

x = tf.keras.layers.Flatten()(vgg16.output)
pred = tf.keras.layers.Dense(64,activation='relu')(x)
pred1 = tf.keras.layers.Dense(3,activation='softmax')(pred)

from tensorflow.keras.models import Model
vggmodel = Model(inputs=vgg16.input,outputs=pred1)
vggmodel.compile(optimizer='adam',loss=tf.keras.losses.SparseCategoricalCrossentropy(),metrics=['accuracy'])
vggmodel.fit(X_train,Y_train,batch_size=32,epochs=10)
vggmodel.evaluate(X_test,Y_test)

58892288/58889256 [==============================] - 1s 0us/step
Epoch 1/10
141/141 [==============================] - 49s 271ms/step - loss: 5.9432 - accuracy: 0.8218
Epoch 2/10
141/141 [==============================] - 32s 226ms/step - loss: 0.2044 - accuracy: 0.9400
Epoch 3/10
141/141 [==============================] - 32s 228ms/step - loss: 0.0814 - accuracy: 0.9716
Epoch 4/10
141/141 [==============================] - 32s 227ms/step - loss: 0.0471 - accuracy: 0.9846
Epoch 5/10
141/141 [==============================] - 32s 227ms/step - loss: 0.0379 - accuracy: 0.9898
Epoch 6/10
141/141 [==============================] - 32s 228ms/step - loss: 0.0515 - accuracy: 0.9871
Epoch 7/10
141/141 [==============================] - 32s 227ms/step - loss: 0.0309 - accuracy: 0.9915
Epoch 8/10
141/141 [==============================] - 32s 228ms/step - loss: 0.0076 - accuracy: 0.9973
Epoch 9/10
141/141 [==============================] - 32s 227ms/step - loss: 0.0529 - accuracy: 0.9846
Epoch 10

[0.1856730580329895, 0.9646666646003723]

InceptionV3

In [ ]:
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications.inception_v3 import preprocess_input
inceptionV3 = InceptionV3(input_shape=(224,224,3),weights='imagenet',include_top=False)

for layer in inceptionV3.layers:
    layer.trainable = False

x = tf.keras.layers.Flatten()(inceptionV3.output)
pred1 = tf.keras.layers.Dense(3,activation='softmax')(x)

from tensorflow.keras.models import Model
inceptionV3model = Model(inputs=inceptionV3.input,outputs=pred1)
inceptionV3model.compile(optimizer='adam',loss=tf.keras.losses.SparseCategoricalCrossentropy(),metrics=['accuracy'])
inceptionV3model.fit(X_train,Y_train,batch_size=32,epochs=10)
inceptionV3model.evaluate(X_test,Y_test)

Epoch 1/10
141/141 [==============================] - 27s 155ms/step - loss: 112.1652 - accuracy: 0.6699
Epoch 2/10
141/141 [==============================] - 22s 154ms/step - loss: 13.6095 - accuracy: 0.8472
Epoch 3/10
141/141 [==============================] - 22s 154ms/step - loss: 8.0355 - accuracy: 0.8847
Epoch 4/10
141/141 [==============================] - 22s 154ms/step - loss: 14.9792 - accuracy: 0.8609
Epoch 5/10
141/141 [==============================] - 22s 154ms/step - loss: 7.3042 - accuracy: 0.9122
Epoch 6/10
141/141 [==============================] - 22s 154ms/step - loss: 8.1841 - accuracy: 0.9115
Epoch 7/10
141/141 [==============================] - 22s 154ms/step - loss: 12.2744 - accuracy: 0.9052
Epoch 8/10
141/141 [==============================] - 22s 154ms/step - loss: 7.7594 - accuracy: 0.9174
Epoch 9/10
141/141 [==============================] - 22s 154ms/step - loss: 9.5349 - accuracy: 0.9193
Epoch 10/10
47/47 [==============================] - 10s 184ms/step 

[28.91088104248047, 0.8939999938011169]

inception_resnet_v2 

In [ ]:
from tensorflow.keras.applications.inception_resnet_v2 import InceptionResNetV2
from tensorflow.keras.applications.inception_resnet_v2 import preprocess_input
inceptionresnetV3 = InceptionResNetV2(input_shape=(224,224,3),weights='imagenet',include_top=False)

for layer in inceptionresnetV3.layers:
    layer.trainable = False

x = tf.keras.layers.Flatten()(inceptionresnetV3.output)
pred1 = tf.keras.layers.Dense(3,activation='softmax')(x)

from tensorflow.keras.models import Model
inceptionresnetV3model = Model(inputs=inceptionresnetV3.input,outputs=pred1)
inceptionresnetV3model.compile(optimizer='adam',loss=tf.keras.losses.SparseCategoricalCrossentropy(),metrics=['accuracy'])
inceptionresnetV3model.fit(X_train,Y_train,batch_size=32,epochs=10)
inceptionresnetV3model.evaluate(X_test,Y_test)

Epoch 1/10
141/141 [==============================] - 67s 381ms/step - loss: 1507.6552 - accuracy: 0.5392
Epoch 2/10
141/141 [==============================] - 53s 378ms/step - loss: 297.9416 - accuracy: 0.7060
Epoch 3/10
141/141 [==============================] - 53s 378ms/step - loss: 322.2433 - accuracy: 0.7403
Epoch 4/10
141/141 [==============================] - 53s 378ms/step - loss: 300.5535 - accuracy: 0.7503
Epoch 5/10
141/141 [==============================] - 53s 377ms/step - loss: 261.5374 - accuracy: 0.7583
Epoch 6/10
141/141 [==============================] - 53s 378ms/step - loss: 249.9082 - accuracy: 0.7712
Epoch 7/10
141/141 [==============================] - 53s 378ms/step - loss: 227.6602 - accuracy: 0.7742
Epoch 8/10
141/141 [==============================] - 53s 378ms/step - loss: 239.2692 - accuracy: 0.7811
Epoch 9/10
141/141 [==============================] - 53s 378ms/step - loss: 286.4159 - accuracy: 0.7664
Epoch 10/10
47/47 [==============================] - 2

[194.36497497558594, 0.8153333067893982]

MobileNet

In [ ]:
from tensorflow.keras.applications.mobilenet import MobileNet
from tensorflow.keras.applications.mobilenet import preprocess_input
mobilenet = MobileNet(input_shape=(224,224,3),weights='imagenet',include_top=False)

for layer in mobilenet.layers:
    layer.trainable = False

x = tf.keras.layers.Flatten()(mobilenet.output)
pred1 = tf.keras.layers.Dense(3,activation='softmax')(x)

from tensorflow.keras.models import Model
mobilenetmodel = Model(inputs=mobilenet.input,outputs=pred1)
mobilenetmodel.compile(optimizer='adam',loss=tf.keras.losses.SparseCategoricalCrossentropy(),metrics=['accuracy'])
mobilenetmodel.fit(X_train,Y_train,batch_size=32,epochs=10)
mobilenetmodel.evaluate(X_test,Y_test)

17227776/17225924 [==============================] - 1s 0us/step
Epoch 1/10
141/141 [==============================] - 12s 72ms/step - loss: 2.4368 - accuracy: 0.7817
Epoch 2/10
141/141 [==============================] - 10s 69ms/step - loss: 0.5724 - accuracy: 0.9066
Epoch 3/10
141/141 [==============================] - 10s 69ms/step - loss: 0.4230 - accuracy: 0.9411
Epoch 4/10
141/141 [==============================] - 10s 69ms/step - loss: 0.1507 - accuracy: 0.9737
Epoch 5/10
141/141 [==============================] - 10s 69ms/step - loss: 0.1685 - accuracy: 0.9715
Epoch 6/10
141/141 [==============================] - 10s 69ms/step - loss: 0.1231 - accuracy: 0.9800
Epoch 7/10
141/141 [==============================] - 10s 69ms/step - loss: 0.0602 - accuracy: 0.9864
Epoch 8/10
141/141 [==============================] - 10s 70ms/step - loss: 0.1577 - accuracy: 0.9751
Epoch 9/10
141/141 [==============================] - 10s 69ms/step - loss: 0.1038 - accuracy: 0.9848
Epoch 10/10
47/47

[0.8796747922897339, 0.9326666593551636]

densenet

In [ ]:
from tensorflow.keras.applications.densenet import DenseNet121
from tensorflow.keras.applications.densenet import preprocess_input
densenet = DenseNet121(input_shape=(224,224,3),weights='imagenet',include_top=False)

for layer in densenet.layers:
    layer.trainable = False

x = tf.keras.layers.Flatten()(densenet.output)
pred1 = tf.keras.layers.Dense(3,activation='softmax')(x)

from tensorflow.keras.models import Model
densenetmodel = Model(inputs=densenet.input,outputs=pred1)
densenetmodel.compile(optimizer='adam',loss=tf.keras.losses.SparseCategoricalCrossentropy(),metrics=['accuracy'])
densenetmodel.fit(X_train,Y_train,batch_size=32,epochs=10)
densenetmodel.evaluate(X_test,Y_test)

29089792/29084464 [==============================] - 0s 0us/step
Epoch 1/10
141/141 [==============================] - 38s 208ms/step - loss: 17.3382 - accuracy: 0.7025
Epoch 2/10
141/141 [==============================] - 28s 199ms/step - loss: 1.3655 - accuracy: 0.9196
Epoch 3/10
141/141 [==============================] - 28s 200ms/step - loss: 1.2282 - accuracy: 0.9251
Epoch 4/10
141/141 [==============================] - 28s 200ms/step - loss: 0.5888 - accuracy: 0.9607
Epoch 5/10
141/141 [==============================] - 28s 199ms/step - loss: 2.1829 - accuracy: 0.9234
Epoch 6/10
141/141 [==============================] - 28s 200ms/step - loss: 0.5016 - accuracy: 0.9704
Epoch 7/10
141/141 [==============================] - 28s 199ms/step - loss: 0.5028 - accuracy: 0.9679
Epoch 8/10
141/141 [==============================] - 28s 199ms/step - loss: 0.5225 - accuracy: 0.9722
Epoch 9/10
141/141 [==============================] - 28s 200ms/step - loss: 0.4254 - accuracy: 0.9800
Epoch 1

[1.9290883541107178, 0.937333345413208]

In [ ]:
Y_pred_With_Our_model = 

In [ ]:
densenetmodel.save('densenet.h5')

In [ ]:
mobilenetmodel.save('mobilenet.h5')

In [ ]:
inceptionresnetV3model.save('inceptionresnetV3model.h5')

In [ ]:
inceptionV3model.save('inceptionV3model.h5')

In [ ]:
vggmodel.save('vggmodel.h5')

In [ ]:
model.save('resnet101.h5')

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report

In [ ]:
Y_pred_With_Our_model = model1.predict(X_test) 

In [ ]:
Y_pred_With_Our_model

array([[2.2441588e-06, 9.9999368e-01, 4.0073623e-06],
       [9.9999917e-01, 5.3510416e-07, 3.8801525e-07],
       [9.9926633e-01, 7.3186675e-04, 1.7866469e-06],
       ...,
       [2.4261595e-05, 9.9962008e-01, 3.5553941e-04],
       [9.0955627e-01, 3.2954668e-03, 8.7148331e-02],
       [1.5670167e-04, 9.9984252e-01, 6.8340415e-07]], dtype=float32)

In [ ]:
rounded_pred=np.argmax(Y_pred_With_Our_model, axis=1)

In [ ]:
print(confusion_matrix(Y_test,rounded_pred))

[[485  15   0]
 [  4 494   2]
 [  3  19 478]]


In [ ]:
print(classification_report(Y_test,rounded_pred))

              precision    recall  f1-score   support

           0       0.99      0.97      0.98       500
           1       0.94      0.99      0.96       500
           2       1.00      0.96      0.98       500

    accuracy                           0.97      1500
   macro avg       0.97      0.97      0.97      1500
weighted avg       0.97      0.97      0.97      1500



In [ ]:
y_pred_dense = densenetmodel.predict(X_test) 
rounded_pred_dense = np.argmax(y_pred_dense, axis=1)
print(confusion_matrix(Y_test,rounded_pred_dense))
print(classification_report(Y_test,rounded_pred_dense))

[[453  37  10]
 [  4 487   9]
 [  3  31 466]]
              precision    recall  f1-score   support

           0       0.98      0.91      0.94       500
           1       0.88      0.97      0.92       500
           2       0.96      0.93      0.95       500

    accuracy                           0.94      1500
   macro avg       0.94      0.94      0.94      1500
weighted avg       0.94      0.94      0.94      1500



In [ ]:
y_pred_mobile = mobilenetmodel.predict(X_test) 
rounded_pred_mobile = np.argmax(y_pred_mobile, axis=1)
print(confusion_matrix(Y_test,rounded_pred_mobile))
print(classification_report(Y_test,rounded_pred_mobile))

[[474  17   9]
 [ 26 457  17]
 [ 11  21 468]]
              precision    recall  f1-score   support

           0       0.93      0.95      0.94       500
           1       0.92      0.91      0.92       500
           2       0.95      0.94      0.94       500

    accuracy                           0.93      1500
   macro avg       0.93      0.93      0.93      1500
weighted avg       0.93      0.93      0.93      1500



In [ ]:
y_pred_incepres = inceptionresnetV3model.predict(X_test) 
rounded_pred_incepres = np.argmax(y_pred_incepres, axis=1)
print(confusion_matrix(Y_test,rounded_pred_incepres))
print(classification_report(Y_test,rounded_pred_incepres))

[[349 114  37]
 [ 27 420  53]
 [ 14  32 454]]
              precision    recall  f1-score   support

           0       0.89      0.70      0.78       500
           1       0.74      0.84      0.79       500
           2       0.83      0.91      0.87       500

    accuracy                           0.82      1500
   macro avg       0.82      0.82      0.81      1500
weighted avg       0.82      0.82      0.81      1500



In [ ]:
y_pred_incev3 = inceptionV3model.predict(X_test) 
rounded_pred_incev3 = np.argmax(y_pred_incev3, axis=1)
print(confusion_matrix(Y_test,rounded_pred_incev3))
print(classification_report(Y_test,rounded_pred_incev3))

[[491   6   3]
 [ 83 391  26]
 [ 23  18 459]]
              precision    recall  f1-score   support

           0       0.82      0.98      0.90       500
           1       0.94      0.78      0.85       500
           2       0.94      0.92      0.93       500

    accuracy                           0.89      1500
   macro avg       0.90      0.89      0.89      1500
weighted avg       0.90      0.89      0.89      1500



In [ ]:
y_pred_vgg = vggmodel.predict(X_test) 
rounded_pred_vgg = np.argmax(y_pred_vgg, axis=1)
print(confusion_matrix(Y_test,rounded_pred_vgg))
print(classification_report(Y_test,rounded_pred_vgg))

[[484  15   1]
 [  5 494   1]
 [  0  24 476]]
              precision    recall  f1-score   support

           0       0.99      0.97      0.98       500
           1       0.93      0.99      0.96       500
           2       1.00      0.95      0.97       500

    accuracy                           0.97      1500
   macro avg       0.97      0.97      0.97      1500
weighted avg       0.97      0.97      0.97      1500



In [ ]:
y_pred_resnet101 = model.predict(X_test) 
rounded_pred_resnet101 = np.argmax(y_pred_resnet101, axis=1)
print(confusion_matrix(Y_test,rounded_pred_resnet101))
print(classification_report(Y_test,rounded_pred_resnet101))

[[456  41   3]
 [  4 492   4]
 [  2  20 478]]
              precision    recall  f1-score   support

           0       0.99      0.91      0.95       500
           1       0.89      0.98      0.93       500
           2       0.99      0.96      0.97       500

    accuracy                           0.95      1500
   macro avg       0.95      0.95      0.95      1500
weighted avg       0.95      0.95      0.95      1500



In [ ]:
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input
def build_model2(hp):
  vgg16 = VGG16(input_shape=(224,224,3),weights='imagenet',include_top=False)

  for layer in vgg16.layers:
    layer.trainable = False

  for index in range(hp.Int('num_conv2D',1,4)):
    x = tf.keras.layers.Conv2D(filters=64,kernel_size=hp.Choice('conv1_kernel',values=[2,3]),padding='same',activation='relu')(vgg16.output)
    x = tf.keras.layers.Dropout(rate=hp.Choice('dropoutinside1',values=[0.1,0.2,0.3]))(x)
    
  x = tf.keras.layers.MaxPooling2D(pool_size=hp.Choice('pooling1',values=[3]),padding = 'same')(x)
  x = tf.keras.layers.Dropout(rate=hp.Choice('dropoutoutside1',values=[0.1,0.2,0.3]))(x)
  
  for index in range(hp.Int('num_conv2D_1',1,4)):
    x = tf.keras.layers.Conv2D(filters=128,kernel_size=hp.Choice('conv2_kernel',values=[2,3]),padding='same',activation='relu')(x)
    x = tf.keras.layers.Dropout(rate=hp.Choice('dropoutinside2',values=[0.1,0.2,0.3]))(x)
   
  x = tf.keras.layers.MaxPooling2D(pool_size=hp.Choice('pooling2',values=[3]),padding = 'same')(x)
  x = tf.keras.layers.Dropout(rate=hp.Choice('dropoutoutside2',values=[0.1,0.2,0.3]))(x)
 
  for index in range(hp.Int('num_conv2D_2',1,4)):
    x = tf.keras.layers.Conv2D(filters=256,kernel_size=hp.Choice('conv3_kernel',values=[2,3]),padding='same',activation='relu')(x)
    x = tf.keras.layers.Dropout(rate=hp.Choice('dropoutinside3',values=[0.1,0.2,0.3]))(x)
  
  x = tf.keras.layers.MaxPooling2D(pool_size=hp.Choice('pooling3',values=[3]),padding = 'same')(x)
  x = tf.keras.layers.Dropout(rate=hp.Choice('dropoutoutside3',values=[0.1,0.2,0.3]))(x)

  x = tf.keras.layers.Flatten()(x)
  for i in range(hp.Int('num_layers', 1, 5)):
    x = tf.keras.layers.Dense(units=hp.Int('units' + str(i),min_value=128,max_value=512,step=32),activation='relu')(x)
    x = tf.keras.layers.Dropout(rate=hp.Choice('dropout3',values=[0.1,0.2,0.3]))(x)
    
  x = tf.keras.layers.Dense(3,activation='softmax')(x)
  
  model = Model(inputs=vgg16.input,outputs=x)
  model.compile(optimizer=tf.keras.optimizers.Adam(hp.Choice('learning_rate',values=[1e-4])),loss=tf.keras.losses.SparseCategoricalCrossentropy(),metrics=['accuracy'])

  return model

In [ ]:
tuner_search5 = RandomSearch(build_model2,objective='val_accuracy',max_trials=5,directory='output6')

In [ ]:
tuner_search5.search(X_train,Y_train,epochs=10,validation_data=(X_test,Y_test))

Trial 5 Complete [00h 07m 20s]
val_accuracy: 0.9466666579246521

Best val_accuracy So Far: 0.9653333425521851
Total elapsed time: 00h 36m 54s
INFO:tensorflow:Oracle triggered exit


In [ ]:
from tensorflow.keras.applications.resnet import ResNet101
from tensorflow.keras.applications.resnet import preprocess_input
def build_model4(hp):
  resnet101 = ResNet101(input_shape=(224,224,3),weights='imagenet',include_top=False)

  for layer in resnet101.layers:
    layer.trainable = False

  for index in range(hp.Int('num_conv2D',1,4)):
    x = tf.keras.layers.Conv2D(filters=64,kernel_size=hp.Choice('conv1_kernel',values=[2,3]),padding='same',activation='relu',kernel_initializer=tf.keras.initializers.HeUniform())(resnet101.output)
    x = tf.keras.layers.Dropout(rate=hp.Choice('dropoutinside1',values=[0.1,0.2,0.3]))(x)
    
  x = tf.keras.layers.MaxPooling2D(pool_size=hp.Choice('pooling1',values=[3]),padding = 'same')(x)
  x = tf.keras.layers.Dropout(rate=hp.Choice('dropoutoutside1',values=[0.1,0.2,0.3]))(x)
  
  for index in range(hp.Int('num_conv2D_1',1,4)):
    x = tf.keras.layers.Conv2D(filters=128,kernel_size=hp.Choice('conv2_kernel',values=[2,3]),padding='same',activation='relu',kernel_initializer=tf.keras.initializers.HeUniform())(x)
    x = tf.keras.layers.Dropout(rate=hp.Choice('dropoutinside2',values=[0.1,0.2,0.3]))(x)
   
  x = tf.keras.layers.MaxPooling2D(pool_size=hp.Choice('pooling2',values=[3]),padding = 'same')(x)
  x = tf.keras.layers.Dropout(rate=hp.Choice('dropoutoutside2',values=[0.1,0.2,0.3]))(x)
 
  for index in range(hp.Int('num_conv2D_2',1,4)):
    x = tf.keras.layers.Conv2D(filters=256,kernel_size=hp.Choice('conv3_kernel',values=[2,3]),padding='same',activation='relu',kernel_initializer=tf.keras.initializers.HeUniform())(x)
    x = tf.keras.layers.Dropout(rate=hp.Choice('dropoutinside3',values=[0.1,0.2,0.3]))(x)
  
  x = tf.keras.layers.MaxPooling2D(pool_size=hp.Choice('pooling3',values=[3]),padding = 'same')(x)
  x = tf.keras.layers.Dropout(rate=hp.Choice('dropoutoutside3',values=[0.1,0.2,0.3]))(x)

  x = tf.keras.layers.Flatten()(x)
  for i in range(hp.Int('num_layers', 1, 5)):
    x = tf.keras.layers.Dense(units=hp.Int('units' + str(i),min_value=128,max_value=512,step=32),activation='relu',kernel_initializer=tf.keras.initializers.HeUniform())(x)
    x = tf.keras.layers.Dropout(rate=hp.Choice('dropout_dense_'+str(i),values=[0.1,0.2,0.3]))(x)
    
  x = tf.keras.layers.Dense(3,activation='softmax')(x)
  
  model = Model(inputs=resnet101.input,outputs=x)
  model.compile(optimizer=tf.keras.optimizers.Adam(hp.Choice('learning_rate',values=[1e-4])),loss=tf.keras.losses.SparseCategoricalCrossentropy(),metrics=['accuracy'])

  return model

In [ ]:
tuner_search6 = RandomSearch(build_model4,objective='val_accuracy',max_trials=10,directory='output7')

In [ ]:
tuner_search6.search(X_train,Y_train,epochs=10,validation_data=(X_test,Y_test))

Trial 2 Complete [00h 11m 23s]
val_accuracy: 0.9620000123977661

Best val_accuracy So Far: 0.9620000123977661
Total elapsed time: 00h 22m 47s

Search: Running Trial #3

Hyperparameter    |Value             |Best Value So Far 
num_conv2D        |2                 |2                 
conv1_kernel      |2                 |2                 
dropoutinside1    |0.3               |0.1               
pooling1          |3                 |3                 
dropoutoutside1   |0.1               |0.2               
num_conv2D_1      |1                 |3                 
conv2_kernel      |2                 |3                 
dropoutinside2    |0.2               |0.1               
pooling2          |3                 |3                 
dropoutoutside2   |0.2               |0.1               
num_conv2D_2      |4                 |3                 
conv3_kernel      |2                 |3                 
dropoutinside3    |0.1               |0.2               
pooling3          |3             

KeyError: ignored

In [ ]:
tuner_search7 = RandomSearch(build_model4,objective='val_accuracy',max_trials=20,directory='output8')

In [ ]:
tuner_search7.search(X_train,Y_train,epochs=10,validation_data=(X_test,Y_test))

Trial 18 Complete [00h 05m 40s]
val_accuracy: 0.940666675567627

Best val_accuracy So Far: 0.9700000286102295
Total elapsed time: 01h 41m 54s

Search: Running Trial #19

Hyperparameter    |Value             |Best Value So Far 
num_conv2D        |2                 |2                 
conv1_kernel      |2                 |3                 
dropoutinside1    |0.1               |0.2               
pooling1          |3                 |3                 
dropoutoutside1   |0.2               |0.3               
num_conv2D_1      |1                 |4                 
conv2_kernel      |2                 |3                 
dropoutinside2    |0.2               |0.1               
pooling2          |3                 |3                 
dropoutoutside2   |0.3               |0.3               
num_conv2D_2      |1                 |4                 
conv3_kernel      |3                 |2                 
dropoutinside3    |0.3               |0.1               
pooling3          |3            

KeyboardInterrupt: ignored

In [ ]:
from google.colab import files

In [ ]:
files.download('densenet.h5')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
files.download('inceptionV3model.h5')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
files.download('inceptionresnetV3model.h5')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
files.download('mobilenet.h5')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
files.download('resnet101.h5')

NameError: ignored

In [ ]:
files.download('vggmodel.h5')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
files.download('best_model.h5')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
from tensorflow.keras.applications.resnet import ResNet101
from tensorflow.keras.applications.resnet import preprocess_input
def build_model5(hp):
  resnet101 = ResNet101(input_shape=(224,224,3),weights='imagenet',include_top=False)

  for layer in resnet101.layers:
    layer.trainable = False

  x = tf.keras.layers.Conv2D(filters=64,kernel_size=2,padding='same',activation='relu',kernel_initializer=tf.keras.initializers.HeUniform())(resnet101.output)
  x = tf.keras.layers.Dropout(rate=hp.Choice('dropoutinside1',values=[0.1,0.2,0.3]))(x)
  x = tf.keras.layers.Conv2D(filters=64,kernel_size=2,padding='same',activation='relu',kernel_initializer=tf.keras.initializers.HeUniform())(resnet101.output)
  x = tf.keras.layers.Dropout(rate=hp.Choice('dropoutinside2',values=[0.1,0.2,0.3]))(x)

  x = tf.keras.layers.MaxPooling2D(pool_size=3,padding = 'same')(x)
  x = tf.keras.layers.Dropout(rate=hp.Choice('dropoutoutside3',values=[0.1,0.2,0.3]))(x)
  
  x = tf.keras.layers.Conv2D(filters=128,kernel_size=2,padding='same',activation='relu',kernel_initializer=tf.keras.initializers.HeUniform())(x)
  x = tf.keras.layers.Dropout(rate=hp.Choice('dropoutinside4',values=[0.1,0.2,0.3]))(x)
  x = tf.keras.layers.Conv2D(filters=128,kernel_size=2,padding='same',activation='relu',kernel_initializer=tf.keras.initializers.HeUniform())(x)
  x = tf.keras.layers.Dropout(rate=hp.Choice('dropoutinside5',values=[0.1,0.2,0.3]))(x)
  x = tf.keras.layers.Conv2D(filters=128,kernel_size=2,padding='same',activation='relu',kernel_initializer=tf.keras.initializers.HeUniform())(x)
  x = tf.keras.layers.Dropout(rate=hp.Choice('dropoutinside6',values=[0.1,0.2,0.3]))(x)
   
  x = tf.keras.layers.MaxPooling2D(pool_size=3,padding = 'same')(x)
  x = tf.keras.layers.Dropout(rate=hp.Choice('dropoutoutside7',values=[0.1,0.2,0.3]))(x)
 
  
  x = tf.keras.layers.Conv2D(filters=256,kernel_size=2,padding='same',activation='relu',kernel_initializer=tf.keras.initializers.HeUniform())(x)
  x = tf.keras.layers.Dropout(rate=hp.Choice('dropoutinside8',values=[0.1,0.2,0.3]))(x)
  x = tf.keras.layers.Conv2D(filters=256,kernel_size=2,padding='same',activation='relu',kernel_initializer=tf.keras.initializers.HeUniform())(x)
  x = tf.keras.layers.Dropout(rate=hp.Choice('dropoutinside9',values=[0.1,0.2,0.3]))(x)
  x = tf.keras.layers.Conv2D(filters=256,kernel_size=2,padding='same',activation='relu',kernel_initializer=tf.keras.initializers.HeUniform())(x)
  x = tf.keras.layers.Dropout(rate=hp.Choice('dropoutinside10',values=[0.1,0.2,0.3]))(x)
  
  x = tf.keras.layers.MaxPooling2D(pool_size=3,padding = 'same')(x)
  x = tf.keras.layers.Dropout(rate=hp.Choice('dropoutoutside3',values=[0.1,0.2,0.3]))(x)

  x = tf.keras.layers.Flatten()(x)
  for i in range(hp.Int('num_layers', 1, 10)):
    x = tf.keras.layers.Dense(units=hp.Int('units' + str(i),min_value=128,max_value=512,step=32),activation='relu',kernel_initializer=tf.keras.initializers.HeUniform())(x)
    x = tf.keras.layers.Dropout(rate=hp.Choice('dropout_dense_'+str(i),values=[0.1,0.2,0.3]))(x)
    
  x = tf.keras.layers.Dense(3,activation='softmax')(x)
  
  model = Model(inputs=resnet101.input,outputs=x)
  model.compile(optimizer=tf.keras.optimizers.Adam(hp.Choice('learning_rate',values=[1e-4])),loss=tf.keras.losses.SparseCategoricalCrossentropy(),metrics=['accuracy'])

  return model

In [ ]:
from tensorflow.keras.applications.resnet import ResNet101
from tensorflow.keras.applications.resnet import preprocess_input
resnet101 = ResNet101(input_shape=(224,224,3),weights='imagenet',include_top=False)

for layer in resnet101.layers:
  layer.trainable = False

x = tf.keras.layers.Conv2D(filters=64,kernel_size=2,padding='same',activation='relu',kernel_initializer=tf.keras.initializers.HeUniform())(resnet101.output)
x = tf.keras.layers.Conv2D(filters=64,kernel_size=2,padding='same',activation='relu',kernel_initializer=tf.keras.initializers.HeUniform())(resnet101.output)
x = tf.keras.layers.Dropout(rate=0.2)(x)

x = tf.keras.layers.MaxPooling2D(pool_size=3,padding = 'same')(x)
x = tf.keras.layers.Dropout(rate=0.3)(x)
  
x = tf.keras.layers.Conv2D(filters=128,kernel_size=2,padding='same',activation='relu',kernel_initializer=tf.keras.initializers.HeUniform())(x)
x = tf.keras.layers.Conv2D(filters=128,kernel_size=2,padding='same',activation='relu',kernel_initializer=tf.keras.initializers.HeUniform())(x)
x = tf.keras.layers.Conv2D(filters=128,kernel_size=2,padding='same',activation='relu',kernel_initializer=tf.keras.initializers.HeUniform())(x)
x = tf.keras.layers.Dropout(rate=0.3)(x)

x = tf.keras.layers.MaxPooling2D(pool_size=3,padding = 'same')(x)
x = tf.keras.layers.Dropout(rate=0.3)(x)
 
x = tf.keras.layers.Conv2D(filters=256,kernel_size=2,padding='same',activation='relu',kernel_initializer=tf.keras.initializers.HeUniform())(x)
x = tf.keras.layers.Conv2D(filters=256,kernel_size=2,padding='same',activation='relu',kernel_initializer=tf.keras.initializers.HeUniform())(x)
x = tf.keras.layers.Conv2D(filters=256,kernel_size=2,padding='same',activation='relu',kernel_initializer=tf.keras.initializers.HeUniform())(x)
x = tf.keras.layers.Dropout(rate=0.2)(x)

x = tf.keras.layers.MaxPooling2D(pool_size=3,padding = 'same')(x)
x = tf.keras.layers.Dropout(rate=0.1)(x)

x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dense(units=512,activation='relu',kernel_initializer=tf.keras.initializers.HeUniform())(x)
x = tf.keras.layers.Dense(units=512,activation='relu',kernel_initializer=tf.keras.initializers.HeUniform())(x)
x = tf.keras.layers.Dense(units=448,activation='relu',kernel_initializer=tf.keras.initializers.HeUniform())(x)
x = tf.keras.layers.Dense(units=192,activation='relu',kernel_initializer=tf.keras.initializers.HeUniform())(x)
    
x = tf.keras.layers.Dense(3,activation='softmax')(x)
  
bst_model = Model(inputs=resnet101.input,outputs=x)
bst_model.compile(optimizer='adam',loss=tf.keras.losses.SparseCategoricalCrossentropy(),metrics=['accuracy'])


In [ ]:
bst_model.fit(X_train,Y_train,batch_size=32,epochs=10)
bst_model.evaluate(X_test,Y_test)

Epoch 1/10
141/141 [==============================] - 32s 182ms/step - loss: 0.8631 - accuracy: 0.6008
Epoch 2/10
141/141 [==============================] - 24s 173ms/step - loss: 0.2425 - accuracy: 0.9154
Epoch 3/10
141/141 [==============================] - 24s 170ms/step - loss: 0.1729 - accuracy: 0.9474
Epoch 4/10
141/141 [==============================] - 25s 174ms/step - loss: 0.1646 - accuracy: 0.9579
Epoch 5/10
141/141 [==============================] - 25s 175ms/step - loss: 0.1271 - accuracy: 0.9622
Epoch 6/10
141/141 [==============================] - 24s 173ms/step - loss: 0.1282 - accuracy: 0.9620
Epoch 7/10
141/141 [==============================] - 24s 172ms/step - loss: 0.0763 - accuracy: 0.9777
Epoch 8/10
141/141 [==============================] - 24s 173ms/step - loss: 0.0700 - accuracy: 0.9741
Epoch 9/10
141/141 [==============================] - 24s 173ms/step - loss: 0.0717 - accuracy: 0.9806
Epoch 10/10
47/47 [==============================] - 10s 169ms/step - los

[0.11271335184574127, 0.9706666469573975]

In [ ]:
from tensorflow.keras.applications.resnet import ResNet101
from tensorflow.keras.applications.resnet import preprocess_input
resnet101 = ResNet101(input_shape=(224,224,3),weights='imagenet',include_top=False)

for layer in resnet101.layers:
  layer.trainable = False

x = tf.keras.layers.Conv2D(filters=64,kernel_size=2,padding='same',activation='relu',kernel_initializer=tf.keras.initializers.HeUniform())(resnet101.output)
x = tf.keras.layers.Conv2D(filters=64,kernel_size=2,padding='same',activation='relu',kernel_initializer=tf.keras.initializers.HeUniform())(resnet101.output)
x = tf.keras.layers.Dropout(rate=0.2)(x)

x = tf.keras.layers.MaxPooling2D(pool_size=3,padding = 'same')(x)
x = tf.keras.layers.Dropout(rate=0.3)(x)
  
x = tf.keras.layers.Conv2D(filters=128,kernel_size=2,padding='same',activation='relu',kernel_initializer=tf.keras.initializers.HeUniform())(x)
x = tf.keras.layers.Conv2D(filters=128,kernel_size=2,padding='same',activation='relu',kernel_initializer=tf.keras.initializers.HeUniform())(x)
x = tf.keras.layers.Conv2D(filters=128,kernel_size=2,padding='same',activation='relu',kernel_initializer=tf.keras.initializers.HeUniform())(x)
x = tf.keras.layers.Dropout(rate=0.1)(x)

x = tf.keras.layers.MaxPooling2D(pool_size=3,padding = 'same')(x)
x = tf.keras.layers.Dropout(rate=0.1)(x)
 
x = tf.keras.layers.Conv2D(filters=256,kernel_size=2,padding='same',activation='relu',kernel_initializer=tf.keras.initializers.HeUniform())(x)
x = tf.keras.layers.Conv2D(filters=256,kernel_size=2,padding='same',activation='relu',kernel_initializer=tf.keras.initializers.HeUniform())(x)
x = tf.keras.layers.Conv2D(filters=256,kernel_size=2,padding='same',activation='relu',kernel_initializer=tf.keras.initializers.HeUniform())(x)
x = tf.keras.layers.Dropout(rate=0.1)(x)

x = tf.keras.layers.MaxPooling2D(pool_size=3,padding = 'same')(x)
x = tf.keras.layers.Dropout(rate=0.1)(x)

x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dense(units=512,activation='relu',kernel_initializer=tf.keras.initializers.HeUniform())(x)
x = tf.keras.layers.Dense(units=512,activation='relu',kernel_initializer=tf.keras.initializers.HeUniform())(x)
x = tf.keras.layers.Dense(units=448,activation='relu',kernel_initializer=tf.keras.initializers.HeUniform())(x)
x = tf.keras.layers.Dense(units=192,activation='relu',kernel_initializer=tf.keras.initializers.HeUniform())(x)

x = tf.keras.layers.Dense(3,activation='softmax')(x)
  
bst_model = Model(inputs=resnet101.input,outputs=x)
bst_model.compile(optimizer='adam',loss=tf.keras.losses.SparseCategoricalCrossentropy(),metrics=['accuracy'])

In [ ]:
bst_model.fit(X_train,Y_train,batch_size=32,epochs=10)
bst_model.evaluate(X_test,Y_test)

Epoch 1/10
141/141 [==============================] - 30s 173ms/step - loss: 0.7627 - accuracy: 0.6541
Epoch 2/10
141/141 [==============================] - 25s 180ms/step - loss: 0.2404 - accuracy: 0.9253
Epoch 3/10
141/141 [==============================] - 24s 172ms/step - loss: 0.1697 - accuracy: 0.9458
Epoch 4/10
141/141 [==============================] - 24s 171ms/step - loss: 0.1665 - accuracy: 0.9532
Epoch 5/10
141/141 [==============================] - 25s 174ms/step - loss: 0.1166 - accuracy: 0.9631
Epoch 6/10
141/141 [==============================] - 24s 174ms/step - loss: 0.0903 - accuracy: 0.9741
Epoch 7/10
141/141 [==============================] - 24s 173ms/step - loss: 0.0812 - accuracy: 0.9769
Epoch 8/10
141/141 [==============================] - 24s 173ms/step - loss: 0.0557 - accuracy: 0.9838
Epoch 9/10
141/141 [==============================] - 24s 173ms/step - loss: 0.0523 - accuracy: 0.9837
Epoch 10/10
47/47 [==============================] - 10s 169ms/step - los

[0.15367621183395386, 0.9713333249092102]

In [ ]:
from tensorflow.keras.applications.resnet import ResNet101
from tensorflow.keras.applications.resnet import preprocess_input
resnet101 = ResNet101(input_shape=(224,224,3),weights='imagenet',include_top=False)

for layer in resnet101.layers:
  layer.trainable = False

x = tf.keras.layers.Conv2D(filters=64,kernel_size=2,padding='same',activation='relu',kernel_initializer=tf.keras.initializers.HeUniform())(resnet101.output)
x = tf.keras.layers.Conv2D(filters=64,kernel_size=2,padding='same',activation='relu',kernel_initializer=tf.keras.initializers.HeUniform())(resnet101.output)

x = tf.keras.layers.MaxPooling2D(pool_size=3,padding = 'same')(x)
  
x = tf.keras.layers.Conv2D(filters=128,kernel_size=2,padding='same',activation='relu',kernel_initializer=tf.keras.initializers.HeUniform())(x)
x = tf.keras.layers.Conv2D(filters=128,kernel_size=2,padding='same',activation='relu',kernel_initializer=tf.keras.initializers.HeUniform())(x)
x = tf.keras.layers.Conv2D(filters=128,kernel_size=2,padding='same',activation='relu',kernel_initializer=tf.keras.initializers.HeUniform())(x)

x = tf.keras.layers.MaxPooling2D(pool_size=3,padding = 'same')(x)
 
x = tf.keras.layers.Conv2D(filters=256,kernel_size=2,padding='same',activation='relu',kernel_initializer=tf.keras.initializers.HeUniform())(x)
x = tf.keras.layers.Conv2D(filters=256,kernel_size=2,padding='same',activation='relu',kernel_initializer=tf.keras.initializers.HeUniform())(x)
x = tf.keras.layers.Conv2D(filters=256,kernel_size=2,padding='same',activation='relu',kernel_initializer=tf.keras.initializers.HeUniform())(x)

x = tf.keras.layers.MaxPooling2D(pool_size=3,padding = 'same')(x)

x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dense(units=512,activation='relu',kernel_initializer=tf.keras.initializers.HeUniform())(x)
x = tf.keras.layers.Dense(units=512,activation='relu',kernel_initializer=tf.keras.initializers.HeUniform())(x)
x = tf.keras.layers.Dense(units=448,activation='relu',kernel_initializer=tf.keras.initializers.HeUniform())(x)
x = tf.keras.layers.Dense(units=192,activation='relu',kernel_initializer=tf.keras.initializers.HeUniform())(x)

x = tf.keras.layers.Dense(3,activation='softmax')(x)
  
bst_model = Model(inputs=resnet101.input,outputs=x)
bst_model.compile(optimizer='adam',loss=tf.keras.losses.SparseCategoricalCrossentropy(),metrics=['accuracy'])

In [ ]:
bst_model.fit(X_train,Y_train,batch_size=32,epochs=10)
bst_model.evaluate(X_test,Y_test)

Epoch 1/10
141/141 [==============================] - 55s 344ms/step - loss: 0.5761 - accuracy: 0.7426
Epoch 2/10
141/141 [==============================] - 49s 345ms/step - loss: 0.1954 - accuracy: 0.9472
Epoch 3/10
141/141 [==============================] - 49s 346ms/step - loss: 0.0871 - accuracy: 0.9766
Epoch 4/10
141/141 [==============================] - 49s 346ms/step - loss: 0.0636 - accuracy: 0.9831
Epoch 5/10
141/141 [==============================] - 49s 346ms/step - loss: 0.0620 - accuracy: 0.9819
Epoch 6/10
141/141 [==============================] - 49s 346ms/step - loss: 0.0274 - accuracy: 0.9926
Epoch 7/10
141/141 [==============================] - 49s 345ms/step - loss: 0.0127 - accuracy: 0.9967
Epoch 8/10
141/141 [==============================] - 49s 346ms/step - loss: 0.0388 - accuracy: 0.9895
Epoch 9/10
141/141 [==============================] - 49s 346ms/step - loss: 0.0379 - accuracy: 0.9885
Epoch 10/10
47/47 [==============================] - 18s 337ms/step - los

[0.28067755699157715, 0.9626666903495789]

In [ ]:
from tensorflow.keras.applications.resnet import ResNet101
from tensorflow.keras.applications.resnet import preprocess_input
resnet101 = ResNet101(input_shape=(224,224,3),weights='imagenet',include_top=False)

for layer in resnet101.layers:
  layer.trainable = False

x = tf.keras.layers.Conv2D(filters=64,kernel_size=2,padding='same',activation='relu',kernel_initializer=tf.keras.initializers.HeUniform())(resnet101.output)
x = tf.keras.layers.Conv2D(filters=64,kernel_size=2,padding='same',activation='relu',kernel_initializer=tf.keras.initializers.HeUniform())(x)
x = tf.keras.layers.Conv2D(filters=64,kernel_size=2,padding='same',activation='relu',kernel_initializer=tf.keras.initializers.HeUniform())(x)
x = tf.keras.layers.Dropout(rate=0.1)(x)

x = tf.keras.layers.MaxPooling2D(pool_size=3,padding = 'same')(x)
x = tf.keras.layers.Dropout(rate=0.1)(x)

x = tf.keras.layers.Conv2D(filters=128,kernel_size=2,padding='same',activation='relu',kernel_initializer=tf.keras.initializers.HeUniform())(x)
x = tf.keras.layers.Conv2D(filters=128,kernel_size=2,padding='same',activation='relu',kernel_initializer=tf.keras.initializers.HeUniform())(x)
x = tf.keras.layers.Conv2D(filters=128,kernel_size=2,padding='same',activation='relu',kernel_initializer=tf.keras.initializers.HeUniform())(x)
x = tf.keras.layers.Dropout(rate=0.1)(x)

x = tf.keras.layers.MaxPooling2D(pool_size=3,padding = 'same')(x)
x = tf.keras.layers.Dropout(rate=0.1)(x)

x = tf.keras.layers.Conv2D(filters=256,kernel_size=2,padding='same',activation='relu',kernel_initializer=tf.keras.initializers.HeUniform())(x)
x = tf.keras.layers.Conv2D(filters=256,kernel_size=2,padding='same',activation='relu',kernel_initializer=tf.keras.initializers.HeUniform())(x)
x = tf.keras.layers.Conv2D(filters=256,kernel_size=2,padding='same',activation='relu',kernel_initializer=tf.keras.initializers.HeUniform())(x)
x = tf.keras.layers.Dropout(rate=0.1)(x)

x = tf.keras.layers.MaxPooling2D(pool_size=3,padding = 'same')(x)
x = tf.keras.layers.Dropout(rate=0.1)(x)

x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dense(units=512,activation='relu',kernel_initializer=tf.keras.initializers.HeUniform())(x)
x = tf.keras.layers.Dense(units=512,activation='relu',kernel_initializer=tf.keras.initializers.HeUniform())(x)
x = tf.keras.layers.Dense(units=448,activation='relu',kernel_initializer=tf.keras.initializers.HeUniform())(x)
x = tf.keras.layers.Dense(units=192,activation='relu',kernel_initializer=tf.keras.initializers.HeUniform())(x)

x = tf.keras.layers.Dense(3,activation='softmax')(x)
  
bst_model = Model(inputs=resnet101.input,outputs=x)
bst_model.compile(optimizer='adam',loss=tf.keras.losses.SparseCategoricalCrossentropy(),metrics=['accuracy'])

In [ ]:
X_train = np.array(X_train)

In [ ]:
bst_model.fit(X_train,Y_train,batch_size=32,epochs=20)
bst_model.evaluate(X_test,Y_test)

Epoch 1/20
141/141 [==============================] - 19s 94ms/step - loss: 0.7881 - accuracy: 0.6279
Epoch 2/20
141/141 [==============================] - 13s 94ms/step - loss: 0.2694 - accuracy: 0.9146
Epoch 3/20
141/141 [==============================] - 13s 94ms/step - loss: 0.1617 - accuracy: 0.9508
Epoch 4/20
141/141 [==============================] - 13s 94ms/step - loss: 0.1245 - accuracy: 0.9641
Epoch 5/20
141/141 [==============================] - 13s 94ms/step - loss: 0.0940 - accuracy: 0.9726
Epoch 6/20
141/141 [==============================] - 13s 94ms/step - loss: 0.1034 - accuracy: 0.9686
Epoch 7/20
141/141 [==============================] - 13s 92ms/step - loss: 0.0483 - accuracy: 0.9865
Epoch 8/20
141/141 [==============================] - 13s 92ms/step - loss: 0.0469 - accuracy: 0.9842
Epoch 9/20
141/141 [==============================] - 13s 92ms/step - loss: 0.0540 - accuracy: 0.9861
Epoch 10/20
141/141 [==============================] - 13s 92ms/step - loss: 0.063

[0.27580103278160095, 0.9573333263397217]

In [ ]:
from tensorflow.keras.applications.resnet import ResNet101
from tensorflow.keras.applications.resnet import preprocess_input
acc = []
for i in range(10):
  resnet101 = ResNet101(input_shape=(224,224,3),weights='imagenet',include_top=False)

  for layer in resnet101.layers:
    layer.trainable = False

  x = tf.keras.layers.Conv2D(filters=64,kernel_size=2,padding='same',activation='relu',kernel_initializer=tf.keras.initializers.HeUniform())(resnet101.output)
  x = tf.keras.layers.Conv2D(filters=64,kernel_size=2,padding='same',activation='relu',kernel_initializer=tf.keras.initializers.HeUniform())(x)
  x = tf.keras.layers.Dropout(rate=0.2)(x)

  x = tf.keras.layers.MaxPooling2D(pool_size=3,padding = 'same')(x)
  x = tf.keras.layers.Dropout(rate=0.3)(x)

  x = tf.keras.layers.Conv2D(filters=128,kernel_size=2,padding='same',activation='relu',kernel_initializer=tf.keras.initializers.HeUniform())(x)
  x = tf.keras.layers.Conv2D(filters=128,kernel_size=2,padding='same',activation='relu',kernel_initializer=tf.keras.initializers.HeUniform())(x)
  x = tf.keras.layers.Conv2D(filters=128,kernel_size=2,padding='same',activation='relu',kernel_initializer=tf.keras.initializers.HeUniform())(x)
  x = tf.keras.layers.Dropout(rate=0.1)(x)

  x = tf.keras.layers.MaxPooling2D(pool_size=3,padding = 'same')(x)
  x = tf.keras.layers.Dropout(rate=0.1)(x)

  x = tf.keras.layers.Conv2D(filters=256,kernel_size=2,padding='same',activation='relu',kernel_initializer=tf.keras.initializers.HeUniform())(x)
  x = tf.keras.layers.Conv2D(filters=256,kernel_size=2,padding='same',activation='relu',kernel_initializer=tf.keras.initializers.HeUniform())(x)
  x = tf.keras.layers.Conv2D(filters=256,kernel_size=2,padding='same',activation='relu',kernel_initializer=tf.keras.initializers.HeUniform())(x)
  x = tf.keras.layers.Dropout(rate=0.1)(x)

  x = tf.keras.layers.MaxPooling2D(pool_size=3,padding = 'same')(x)
  x = tf.keras.layers.Dropout(rate=0.1)(x)

  x = tf.keras.layers.Flatten()(x)
  x = tf.keras.layers.Dense(units=512,activation='relu',kernel_initializer=tf.keras.initializers.HeUniform())(x)
  x = tf.keras.layers.Dense(units=512,activation='relu',kernel_initializer=tf.keras.initializers.HeUniform())(x)
  x = tf.keras.layers.Dense(units=448,activation='relu',kernel_initializer=tf.keras.initializers.HeUniform())(x)
  x = tf.keras.layers.Dense(units=192,activation='relu',kernel_initializer=tf.keras.initializers.HeUniform())(x)

  x = tf.keras.layers.Dense(3,activation='softmax')(x)
    
  bst_model = Model(inputs=resnet101.input,outputs=x)
  opt = keras.optimizers.RMSprop(learning_rate=0.0001)
  bst_model.compile(optimizer=opt,loss=tf.keras.losses.SparseCategoricalCrossentropy(),metrics=['accuracy'])
  bst_model.fit(X_train,Y_train,batch_size=32,epochs=10)
  acc.append(bst_model.evaluate(X_test,Y_test))
print(acc)
  

Epoch 1/10
141/141 [==============================] - 30s 168ms/step - loss: 0.8947 - accuracy: 0.5597
Epoch 2/10
141/141 [==============================] - 23s 166ms/step - loss: 0.2931 - accuracy: 0.8876
Epoch 3/10
141/141 [==============================] - 24s 169ms/step - loss: 0.2089 - accuracy: 0.9233
Epoch 4/10
141/141 [==============================] - 24s 169ms/step - loss: 0.1672 - accuracy: 0.9401
Epoch 5/10
141/141 [==============================] - 24s 168ms/step - loss: 0.1504 - accuracy: 0.9500
Epoch 6/10
141/141 [==============================] - 24s 168ms/step - loss: 0.1182 - accuracy: 0.9602
Epoch 7/10
141/141 [==============================] - 24s 169ms/step - loss: 0.1045 - accuracy: 0.9675
Epoch 8/10
141/141 [==============================] - 24s 170ms/step - loss: 0.0905 - accuracy: 0.9699
Epoch 9/10
141/141 [==============================] - 24s 169ms/step - loss: 0.0665 - accuracy: 0.9762
Epoch 10/10
47/47 [==============================] - 10s 182ms/step - los

In [ ]:
bst_model.fit(X_train,Y_train,batch_size=1,epochs=10)
bst_model.evaluate(X_test,Y_test)

Epoch 1/10
4500/4500 [==============================] - 92s 19ms/step - loss: 0.7048 - accuracy: 0.7111
Epoch 2/10
4500/4500 [==============================] - 87s 19ms/step - loss: 0.3791 - accuracy: 0.9180
Epoch 3/10
4500/4500 [==============================] - 87s 19ms/step - loss: 0.3924 - accuracy: 0.9345
Epoch 4/10
4500/4500 [==============================] - 86s 19ms/step - loss: 0.5896 - accuracy: 0.9352
Epoch 5/10
4500/4500 [==============================] - 87s 19ms/step - loss: 0.4154 - accuracy: 0.9512
Epoch 6/10
4500/4500 [==============================] - 86s 19ms/step - loss: 0.5444 - accuracy: 0.9566
Epoch 7/10
4500/4500 [==============================] - 86s 19ms/step - loss: 0.6558 - accuracy: 0.9596
Epoch 8/10
4500/4500 [==============================] - 85s 19ms/step - loss: 0.3753 - accuracy: 0.9653
Epoch 9/10
4500/4500 [==============================] - 85s 19ms/step - loss: 0.5360 - accuracy: 0.9629
Epoch 10/10
47/47 [==============================] - 6s 89ms/ste

[0.391681969165802, 0.9340000152587891]

In [ ]:
from tensorflow.keras.applications.resnet import ResNet101
from tensorflow.keras.applications.resnet import preprocess_input
resnet101 = ResNet101(input_shape=(224,224,3),weights='imagenet',include_top=False)

for layer in resnet101.layers:
  layer.trainable = False

x = tf.keras.layers.Conv2D(filters=64,kernel_size=2,padding='same',activation='relu',kernel_initializer=tf.keras.initializers.HeUniform())(resnet101.output)
x = tf.keras.layers.Conv2D(filters=64,kernel_size=2,padding='same',activation='relu',kernel_initializer=tf.keras.initializers.HeUniform())(x)
x = tf.keras.layers.Dropout(rate=0.2)(x)

x = tf.keras.layers.MaxPooling2D(pool_size=3,padding = 'same')(x)
x = tf.keras.layers.Dropout(rate=0.3)(x)

x = tf.keras.layers.Conv2D(filters=128,kernel_size=2,padding='same',activation='relu',kernel_initializer=tf.keras.initializers.HeUniform())(x)
x = tf.keras.layers.Conv2D(filters=128,kernel_size=2,padding='same',activation='relu',kernel_initializer=tf.keras.initializers.HeUniform())(x)
x = tf.keras.layers.Conv2D(filters=128,kernel_size=2,padding='same',activation='relu',kernel_initializer=tf.keras.initializers.HeUniform())(x)
x = tf.keras.layers.Dropout(rate=0.1)(x)

x = tf.keras.layers.MaxPooling2D(pool_size=3,padding = 'same')(x)
x = tf.keras.layers.Dropout(rate=0.1)(x)

x = tf.keras.layers.Conv2D(filters=256,kernel_size=2,padding='same',activation='relu',kernel_initializer=tf.keras.initializers.HeUniform())(x)
x = tf.keras.layers.Conv2D(filters=256,kernel_size=2,padding='same',activation='relu',kernel_initializer=tf.keras.initializers.HeUniform())(x)
x = tf.keras.layers.Conv2D(filters=256,kernel_size=2,padding='same',activation='relu',kernel_initializer=tf.keras.initializers.HeUniform())(x)
x = tf.keras.layers.Dropout(rate=0.1)(x)

x = tf.keras.layers.MaxPooling2D(pool_size=3,padding = 'same')(x)
x = tf.keras.layers.Dropout(rate=0.1)(x)

x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dense(units=128,activation='relu',kernel_initializer=tf.keras.initializers.HeUniform())(x)
x = tf.keras.layers.Dense(units=64,activation='relu',kernel_initializer=tf.keras.initializers.HeUniform())(x)
x = tf.keras.layers.Dense(units=32,activation='relu',kernel_initializer=tf.keras.initializers.HeUniform())(x)

x = tf.keras.layers.Dense(3,activation='softmax')(x)
  
bst_model = Model(inputs=resnet101.input,outputs=x)
opt = keras.optimizers.RMSprop(learning_rate=0.0001)
bst_model.compile(optimizer=opt,loss=tf.keras.losses.SparseCategoricalCrossentropy(),metrics=['accuracy'])

In [ ]:
bst_model.fit(X_train,Y_train,batch_size=32,epochs=10)
bst_model.evaluate(X_test,Y_test)

Epoch 1/10
141/141 [==============================] - 19s 92ms/step - loss: 0.8896 - accuracy: 0.5584
Epoch 2/10
141/141 [==============================] - 13s 92ms/step - loss: 0.3031 - accuracy: 0.8888
Epoch 3/10
141/141 [==============================] - 13s 92ms/step - loss: 0.2183 - accuracy: 0.9173
Epoch 4/10
141/141 [==============================] - 13s 92ms/step - loss: 0.1585 - accuracy: 0.9476
Epoch 5/10
141/141 [==============================] - 13s 92ms/step - loss: 0.1389 - accuracy: 0.9565
Epoch 6/10
141/141 [==============================] - 13s 92ms/step - loss: 0.1179 - accuracy: 0.9613
Epoch 7/10
141/141 [==============================] - 13s 92ms/step - loss: 0.0812 - accuracy: 0.9713
Epoch 8/10
141/141 [==============================] - 13s 92ms/step - loss: 0.0862 - accuracy: 0.9735
Epoch 9/10
141/141 [==============================] - 13s 92ms/step - loss: 0.0680 - accuracy: 0.9779
Epoch 10/10
47/47 [==============================] - 6s 89ms/step - loss: 0.2022 -

[0.20218342542648315, 0.9453333616256714]

In [ ]:
from tensorflow.keras.applications.resnet import ResNet101
from tensorflow.keras.applications.resnet import preprocess_input
resnet101 = ResNet101(input_shape=(224,224,3),weights='imagenet',include_top=False)

for layer in resnet101.layers:
  layer.trainable = False

feature_extractor = resnet101.predict(X_train)

features = feature_extractor.reshape(feature_extractor.shape[0],-1)

x_for_training = features




AlreadyExistsError: ignored

In [ ]:
feature_extractor = resnet101.predict(X_test)

features = feature_extractor.reshape(feature_extractor.shape[0],-1)

x_for_testing = features

In [ ]:
import xgboost as xgb
xgbmodel = xgb.XGBClassifier()
xgbmodel.fit(x_for_training,Y_train)
xgbmodel.score(x_for_testing,Y_test)

In [ ]:
tuner_search8 = RandomSearch(build_model5,objective='val_accuracy',max_trials=20,directory='output9')

In [ ]:
tuner_search8.search(X_train,Y_train,epochs=10,validation_data=(X_test,Y_test))

Trial 5 Complete [00h 05m 40s]
val_accuracy: 0.6620000004768372

Best val_accuracy So Far: 0.9613333344459534
Total elapsed time: 00h 28m 28s

Search: Running Trial #6

Hyperparameter    |Value             |Best Value So Far 
dropoutinside1    |0.2               |0.1               
dropoutinside2    |0.3               |0.1               
dropoutoutside3   |0.2               |0.1               
dropoutinside4    |0.2               |0.1               
dropoutinside5    |0.3               |0.1               
dropoutinside6    |0.3               |0.3               
dropoutoutside7   |0.2               |0.2               
dropoutinside8    |0.1               |0.1               
dropoutinside9    |0.3               |0.2               
dropoutinside10   |0.3               |0.1               
num_layers        |4                 |4                 
units0            |512               |256               
dropout_dense_0   |0.3               |0.3               
learning_rate     |0.0001        